In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import math

model = torch.jit.load("traced_model.pt")
model.eval() 

window = 30
data = torch.load('data/cleaned.pt')

prices = data['SPDR'].to_numpy()
num_train = math.floor(prices.shape[0]*0.8-window)
prices = prices[num_train:]

inputs = data[['Gold', 'SPX', 'DXY', 't', 'CPI_pct', 'Volume', 'Fed Rate']].to_numpy()
test_input = torch.from_numpy(inputs[num_train:, :])
test_input = torch.stack([test_input[i:i+window] for i in range(test_input.size(0) - window)])

funds = 0
correct_up = 0
correct_down = 0
up = 0
down = 0
with torch.no_grad(): 
    test_output = model(test_input)
    returns = test_output[:, -1, 0]
    
    for i in range(0,len(returns)-1):
        today = 30+i
        pct = returns[i]*0.128900121790875
        if pct < 0:
            up+=1
            units_bought = 1000/prices[today]
            profit = units_bought*prices[today+1] - 1000
            funds += profit
            if prices[today+1] > prices[today]:
                correct_up+=1
#         else:
#             down+=1
#             units_sold = 1000/prices[today]
#             funds -= units_sold*prices[today+1] + 1000
#             if prices[today+1] < prices[today]:
#                 correct_down+=1
                
print(funds)
print(correct_up, up)
print(correct_down, down)
            

315.32308611558744
660 1729
0 0


In [24]:
funds = 5000
wrong = 0
total = 0
units = 0

new_input_data = torch.randn(10000, 30, 7)  # New example input data
with torch.no_grad():  # Disable gradient computation
    new_output = model(test_input)
    print(new_output.size())  # Should print [10000, 30, 1]

for day in range(1, test_input.shape[0]-window-1):
# for day in range(1, 50):
    current_input = test_input[day].unsqueeze(0)
#     print(current_input.shape) # 1, 30, 7
    pred = model(current_input)
    item = pred[:, -1].item()
    predicted_return = item*max_gold*100+1    
    open = prices[day+29]
    sign = '+'
    if (prices[day+30] < prices[day+29]):
        sign = '-'
#     print(sign, f"{predicted_return:.3f}", open, prices[day+28])
    predicted_close = predicted_return*open
    if units != 0:
        funds += units*open
        units = 0

    if predicted_return > 0:
        funds -= 1000
        units += 1000/open
        if sign == '-':
            wrong += 1
    elif predicted_return < 0:
        funds += 1000
        units -= 1000/open
        if sign == '+':
            wrong +=1
    total +=1
#     print(position, units, funds)
#     predicted_close = predicted_return*prices[day+29]

if units != 0:
    funds += units*open
    units = 0
print(position, units, funds)
print(wrong, total)

torch.Size([3006, 30, 1])


NameError: name 'max_gold' is not defined